# Adapting 5.0.5
Currently model 5.0.5 is by far the best performing model. We want to see if we can beat it - in any - or all metrics! We will adapt some of the code used in the Higgs hunting dataset.

In [1]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Model Configuration
UNITS = 2 ** 6 # 64
ACTIVATION = 'relu'
DROPOUT = 0.1
NUM_CLASSES = 2

# Training Configuration
BATCH_SIZE_PER_REPLICA = 2 ** 10 # powers of 128 are best

CSV_HEADER = ['Lb_PT', 'Lb_IPCHI2_OWNPV',' Lb_ENDVERTEX_CHI2', 'Lb_HOP', 'LStar_ORIVX_CHI2', 'LStar_DIRA_OWNPV',
              'JPs_FD_ORIVX', 'p_ETA', 'K_ETA', 'L1_ETA', 'L2_ETA', 'Lb_IP01', 'Lb_IP23', 'Lb_IP_OWNPV',
              'p_TRACK_VeloCHI2NDOF', 'ABS_ARTANH_PZ_P', 'MAG_ARSINH_PY_PT', 'SUM_CONE_ISO', 'SUM_LIPCHI2', 
              'LB_TRACKISO', 'LN_COS_THETA', 'LN_LB_MINIPCHI2', 'LN_COS_LBDIRA', 'LN_JPs_DIRA_TOPPV', 'category']

In [3]:
TARGET_FEATURE_NAME = 'category'


FEATURE_NAMES = ['Lb_PT', 'Lb_IPCHI2_OWNPV',' Lb_ENDVERTEX_CHI2', 'Lb_HOP', 'LStar_ORIVX_CHI2', 'LStar_DIRA_OWNPV',
              'JPs_FD_ORIVX', 'p_ETA', 'K_ETA', 'L1_ETA', 'L2_ETA', 'Lb_IP01', 'Lb_IP23', 'Lb_IP_OWNPV',
              'p_TRACK_VeloCHI2NDOF', 'ABS_ARTANH_PZ_P', 'MAG_ARSINH_PY_PT', 'SUM_CONE_ISO', 'SUM_LIPCHI2', 
              'LB_TRACKISO', 'LN_COS_THETA', 'LN_LB_MINIPCHI2', 'LN_COS_LBDIRA', 'LN_JPs_DIRA_TOPPV']

# Model

In [4]:
def get_dataset_from_csv(csv_file_path, batch_size, shuffle=False):

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        column_defaults=[tf.float32 for i in CSV_HEADER],
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=True,
        shuffle=shuffle,
    )
    return dataset.cache()

In [5]:
learning_rate = 0.001
dropout_rate = 0.1
batch_size = 265
num_epochs = 50

hidden_units = [32, 32]


def run_experiment(model):

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy()],
    )

    train_dataset = get_dataset_from_csv('data_5.0.5/train_data.csv', batch_size, shuffle=True)

    test_dataset = get_dataset_from_csv('data_5.0.5/test_data.csv', batch_size)

    print("Start training the model...")
    history = model.fit(train_dataset, epochs=num_epochs)
    print("Model training finished")

    _, accuracy = model.evaluate(test_dataset, verbose=0)

    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    return model

# Create Inputs

In [6]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(), dtype=tf.float32
        )
    return inputs

In [7]:
def encode_inputs(inputs):
    encoded_features = []
    for feature_name in inputs:
        encoded_feature = tf.expand_dims(inputs[feature_name], -1)
        encoded_features.append(encoded_feature)
    all_features = layers.concatenate(encoded_features)
    return all_features

In [8]:
def create_baseline_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs)

    for units in hidden_units:
        features = layers.Dense(units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.ReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1, activation="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


baseline_model = create_baseline_model()
keras.utils.plot_model(baseline_model, show_shapes=True, rankdir="LR")

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


2022-02-25 10:29:14.720479: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-25 10:29:14.725274: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
run_experiment(baseline_model)

Start training the model...
Epoch 1/50


2022-02-25 10:29:15.262513: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-25 10:29:15.275686: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2400145000 Hz


83/83 [==============================] - 3s 13ms/step - loss: 0.4707 - binary_accuracy: 0.7837
Epoch 2/50
83/83 [==============================] - 1s 11ms/step - loss: 0.2616 - binary_accuracy: 0.8983
Epoch 3/50
83/83 [==============================] - 1s 8ms/step - loss: 0.2312 - binary_accuracy: 0.9079
Epoch 4/50
83/83 [==============================] - 1s 13ms/step - loss: 0.2162 - binary_accuracy: 0.9141
Epoch 5/50
83/83 [==============================] - 1s 8ms/step - loss: 0.2061 - binary_accuracy: 0.9187
Epoch 6/50
83/83 [==============================] - 1s 8ms/step - loss: 0.2009 - binary_accuracy: 0.9212
Epoch 7/50
83/83 [==============================] - 1s 7ms/step - loss: 0.1960 - binary_accuracy: 0.9212
Epoch 8/50
83/83 [==============================] - 1s 7ms/step - loss: 0.1920 - binary_accuracy: 0.9231
Epoch 9/50
83/83 [==============================] - 1s 7ms/step - loss: 0.1896 - binary_accuracy: 0.9248
Epoch 10/50
83/83 [==============================] - 0s 5ms/ste

In [10]:
def create_wide_and_deep_model():

    inputs = create_model_inputs()
    wide = encode_inputs(inputs)
    wide = layers.BatchNormalization()(wide)

    deep = encode_inputs(inputs)
    for units in hidden_units:
        deep = layers.Dense(units)(deep)
        deep = layers.BatchNormalization()(deep)
        deep = layers.ReLU()(deep)
        deep = layers.Dropout(dropout_rate)(deep)

    merged = layers.concatenate([wide, deep])
    outputs = layers.Dense(units=1, activation="sigmoid")(merged)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


wide_and_deep_model = create_wide_and_deep_model()
keras.utils.plot_model(wide_and_deep_model, show_shapes=True, rankdir="LR")

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [11]:
run_experiment(wide_and_deep_model)

Start training the model...
Epoch 1/50
83/83 [==============================] - 3s 17ms/step - loss: 0.4498 - binary_accuracy: 0.7885
Epoch 2/50
83/83 [==============================] - 1s 7ms/step - loss: 0.2523 - binary_accuracy: 0.9033
Epoch 3/50
83/83 [==============================] - 1s 7ms/step - loss: 0.2211 - binary_accuracy: 0.9120
Epoch 4/50
83/83 [==============================] - 1s 7ms/step - loss: 0.2093 - binary_accuracy: 0.9172
Epoch 5/50
83/83 [==============================] - 1s 7ms/step - loss: 0.2011 - binary_accuracy: 0.9202
Epoch 6/50
83/83 [==============================] - 1s 7ms/step - loss: 0.1967 - binary_accuracy: 0.9215
Epoch 7/50
83/83 [==============================] - 1s 6ms/step - loss: 0.1919 - binary_accuracy: 0.9240
Epoch 8/50
83/83 [==============================] - 1s 7ms/step - loss: 0.1893 - binary_accuracy: 0.9254
Epoch 9/50
83/83 [==============================] - 1s 10ms/step - loss: 0.1867 - binary_accuracy: 0.9261
Epoch 10/50
83/83 [======

In [12]:
def create_deep_and_cross_model():

    inputs = create_model_inputs()
    x0 = encode_inputs(inputs)

    cross = x0
    for _ in hidden_units:
        units = cross.shape[-1]
        x = layers.Dense(units)(cross)
        cross = x0 * x + cross
    cross = layers.BatchNormalization()(cross)

    deep = x0
    for units in hidden_units:
        deep = layers.Dense(units)(deep)
        deep = layers.BatchNormalization()(deep)
        deep = layers.ReLU()(deep)
        deep = layers.Dropout(dropout_rate)(deep)

    merged = layers.concatenate([cross, deep])
    outputs = layers.Dense(units=1, activation="sigmoid")(merged)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


deep_and_cross_model = create_deep_and_cross_model()
keras.utils.plot_model(deep_and_cross_model, show_shapes=True, rankdir="LR")

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [13]:
deep_cross_model = run_experiment(deep_and_cross_model)

Start training the model...
Epoch 1/50
83/83 [==============================] - 3s 18ms/step - loss: 0.4091 - binary_accuracy: 0.8260
Epoch 2/50
83/83 [==============================] - 1s 8ms/step - loss: 0.2454 - binary_accuracy: 0.9063
Epoch 3/50
83/83 [==============================] - 1s 7ms/step - loss: 0.2117 - binary_accuracy: 0.9173
Epoch 4/50
83/83 [==============================] - 1s 8ms/step - loss: 0.1996 - binary_accuracy: 0.9219
Epoch 5/50
83/83 [==============================] - 1s 8ms/step - loss: 0.1923 - binary_accuracy: 0.9244
Epoch 6/50
83/83 [==============================] - 1s 7ms/step - loss: 0.1863 - binary_accuracy: 0.9262
Epoch 7/50
83/83 [==============================] - 1s 8ms/step - loss: 0.1835 - binary_accuracy: 0.9261
Epoch 8/50
83/83 [==============================] - 1s 8ms/step - loss: 0.1799 - binary_accuracy: 0.9279
Epoch 9/50
83/83 [==============================] - 1s 9ms/step - loss: 0.1775 - binary_accuracy: 0.9294
Epoch 10/50
83/83 [=======